In [1]:
# !pip install transformers datasets accelerate --quiet
# !pip install torchvision --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 32.5 MB/s eta 0:00:0000:0100:01m
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigfram

In [1]:
import os
from PIL import Image
import numpy as np
from datasets import Dataset
import pandas as pd
from transformers import TrOCRProcessor
dir_path = '/kaggle/input/groundtruthtext'
train_path = dir_path + "/" + 'train/gt_text'

2025-06-06 11:42:33.333527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749210153.682781      98 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749210153.769571      98 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_df = pd.read_csv(dir_path + "/" + "train.csv")
train_df['image'] = train_df['image'].apply(lambda x: dir_path + "/" + x)
test_df = pd.read_csv(dir_path + "/" + "test.csv")
test_df['image'] = test_df['image'].apply(lambda x: dir_path + "/" + x)

train_df.head()
dataset = Dataset.from_pandas(train_df)

In [3]:
import torch

def preprocess(example):
    image = Image.open(example["image"]).convert("RGB")

    # Process image
    pixel_values = processor(images=image, return_tensors="pt").pixel_values[0].half()
    example["pixel_values"] = pixel_values.tolist()  # Convert tensor to list

    # Ensure text is not None
    if example.get("text") is None:
        example["text"] = ""

    # Tokenize text
    input_ids = processor.tokenizer(example["text"], return_tensors="pt").input_ids[0]
    example["labels"] = input_ids.tolist()

    return example

In [4]:
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten", use_fast=True)

In [ ]:
for i in range(0, 10000, 500):
    sub = dataset.select(range(i, min(i+500, len(dataset))))
    processed = sub.map(preprocess)
    processed.save_to_disk(f"/kaggle/working/processed_part_{i//500}")

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/4 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/4 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [10]:
from datasets import load_from_disk, concatenate_datasets
n_parts = 10
parts = [load_from_disk(f"/kaggle/working/processed_part_{i}") for i in range(n_parts)]
final_dataset = concatenate_datasets(parts)

In [12]:
final_dataset.save_to_disk(f"/kaggle/working/final")

Saving the dataset (0/38 shards):   0%|          | 0/10500 [00:00<?, ? examples/s]

OSError: [Errno 28] No space left on device

In [11]:
final_dataset

Dataset({
    features: ['image', 'text', 'pixel_values', 'labels'],
    num_rows: 10500
})

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    processor.tokenizer, 
    model=None,  # Set later after model is loaded
    padding=True, 
    return_tensors="pt"
)